In [ ]:
%matplotlib inline
import math
import matplotlib.pyplot as plt
from sklearn import tree
from db import connection, engine
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc
import pandas as pd
import numpy as np
import helpers as fdn
import pixiedust

In [ ]:
select_an = (
    "anbieter.anbieter_id, "
    "anbieter.anbieter_plz, "
    "zuschlag.preis, " # Werden Zuschläge je relevant?? Die haben wir ja nur zur Model Bildung, aber dann nie mehr
    "zuschlag.gatt_wto, "
    "zuschlag.anzahl_angebote, " 
    "anbieter.institution as anbieter_insitution, "
    "cpv_dokument.cpv_nummer as anbieter_cpv, "
    "ausschreibung.meldungsnummer"
)
# anbieter_CPV are all the CPVs the Anbieter ever won a procurement for. So all the CPVs they are interested in. 
select_aus = (
    "anbieter.anbieter_id, "
    "auftraggeber.institution as beschaffungsstelle_institution, "
    "auftraggeber.beschaffungsstelle_plz, "
    "ausschreibung.gatt_wto, "
    "cpv_dokument.cpv_nummer as ausschreibung_cpv, "
    "ausschreibung.meldungsnummer"
)

In [ ]:
data_an = fdn.getFromSimap(select_an)
data_aus = fdn.getFromSimap(select_aus)

In [ ]:
data_an.head()

## Ausschreibung: Anzahl Angebote (done)
Not numerical are set to 0

In [ ]:
# TODO: Check why so many have to be set to 0. (Currently 8012)

# Datacleaning for anzahl_angebote finished like this.
data_an[['anzahl_angebote']] = data_an[['anzahl_angebote']].applymap(fdn.tonumeric)

## Ausschreibung: Preis (done)
Kategorien von jeweils 100'000?

In [ ]:
data_an[data_an['preis'] > 1000000]

In [ ]:
tmp = data_an[data_an['preis']<1000000000]
plt.plot(tmp['preis'].value_counts())
plt.ylabel('Occurences')
plt.xlabel('Price < 1 Bn (* 10^8)')
plt.show()

In [ ]:
# TODO: NOT WORKING --> DELETE

# Map prices to 12 categories
def createPriceCategory(v):
    try:
        v = int(v)
    except:
        v = 0
    print(v)
    switcher = {
        (v <= 100000): 1,
        ((v > 100000) and (v <= 250000)): 2,
        ((v > 250000) and (v <= 500000)): 3,
        ((v > 500000) and (v <= 750000)): 4,
        ((v > 750000) and (v <= 1000000)): 5,
        ((v > 1000000) and (v <= 5000000)): 6,
        ((v > 5000000) and (v <= 10000000)): 7,
        ((v > 10000000) and (v <= 25000000)): 8,
        ((v > 25000000) and (v <= 50000000)): 9,
        ((v > 50000000) and (v <= 100000000)): 10,
        ((v > 100000000) and (v <= 200000000)): 11,
        ((v > 200000000) and (v <= 500000000)): 12,
        (v > 500000000): 13,
    }
    return switcher.get(v, 0)

In [ ]:
# TODO: Kategorien mit Matthias absprechen

# Map prices to 16 categories
def createPriceCategory(v):
    try:
        v = int(v)
    except:
        v = -1
    if v == 0:
        return 0
    if 0 < v <= 100000:
        return 1
    if 100000 < v <= 250000:
        return 2
    if 250000 < v <= 500000:
        return 3
    if 500000 < v <= 750000:
        return 4
    if 750000 < v <= 1000000:
        return 5
    if 1000000 < v <= 2500000:
        return 6
    if 2500000 < v <= 5000000:
        return 7
    if 5000000 < v <= 10000000:
        return 8
    if 10000000 < v <= 25000000:
        return 9
    if 25000000 < v <= 50000000:
        return 10
    if 50000000 < v <= 100000000:
        return 11
    if 100000000 < v <= 200000000:
        return 12
    if 200000000 < v <= 500000000:
        return 13
    if v > 500000000:
        return 14
    else:
        return -1

In [ ]:
data_an[['preis_category']] = data_an[['preis']].applymap(createPriceCategory)

In [ ]:
data_an.preis_category.value_counts()

In [ ]:
data_an.preis.describe()

## Ausschreibung: Gatt WTO (done)
Set everything to either "Yes", "No" or "?"

In [ ]:
data_an.gatt_wto.value_counts()

In [ ]:
def unifyGattWto(val):
    switcher = {
        'Ja': 'Yes',
        'Sì': 'Yes',
        'Nein': 'No',
        'Nei': 'No'
    }
    return switcher.get(val, "?")

In [ ]:
data_an[['gatt_wto']] = data_an[['gatt_wto']].applymap(unifyGattWto)

In [ ]:
data_an.gatt_wto.value_counts()

## Next